## Create Agents to Research and Write an Article

Install the below libraries with their associated version:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [40]:
# warnings
import warnings
warnings.filterwarnings('ignore')

Let's import Crewai libraries with its three main components

In [41]:
# import Crewai libraries
from crewai import Agent, Task, Crew

Import helper functions from our scripts module

In [42]:
import os
import sys

# parent directory path of the package/module "scripts" 
scripts_dir = os.path.abspath("../")
# check and append "scripts_dir" in sys.path if it ain't.
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir) 

from scripts import utils

FileNotFoundError: [Errno 2] No such file or directory

Let's set our LLM

In [ ]:
openai_api_key = utils.get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"

#### Creating Agents
* Let's define our Agent and provide a `role`, `goal`, and `backstory`
* It has been seen that LLMs perform better when they are role playing.

We will create three agents namely: 
* Planner
* Writer
* Editor

##### Agent: Planner

In [5]:
planner = Agent(
    role= "Content Planner",
    goal= "Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article"
              "about the topic {topic}."
              "You collect information that helps the" 
              "audiance learn something"
              "and make informed decisions"
              "Your work is the basis for the Content Writer"
              "to write an article on this topic.",
    allow_delegation=False,
    verbose= True              
)

In [6]:
planner

Agent(role=Content Planner, goal=Plan engaging and factually accurate content on {topic}, backstory=You're working on planning a blog articleabout the topic {topic}.You collect information that helps theaudiance learn somethingand make informed decisionsYour work is the basis for the Content Writerto write an article on this topic.)

##### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [8]:
writer

Agent(role=Content Writer, goal=Write insightful and factually accurate opinion piece about the topic: {topic}, backstory=You're working on a writing a new opinion piece about the topic: {topic}. You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic. You follow the main objectives and direction of the outline, as provide by the Content Planner. You also provide objective and impartial insights and back them up with information provide by the Content Planner. You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.)

##### Agent: Editor

In [9]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

#### Creating Tasks
* Let's define our Tasks and provide them a `description`, `expected_output`, and `agent`

##### Task: Plan

In [10]:
plan = Task(
    description= (
        "1. Prioritize the latest trends, key players, and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline including an introduction, key points,"
            "and a call to action. \n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document with "
                    "with an outline, audience analysis "
                    "SEO keywords, and resources",
    agent= planner,
)

In [11]:
plan

Task(description=1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points,and a call to action. 
4. Include SEO keywords and relevant data or sources., expected_output=A comprehensive content plan document with with an outline, audience analysis SEO keywords, and resources)

##### Task: write

In [12]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [13]:
write

Task(description=1. Use the content plan to craft a compelling blog post on {topic}.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are properly named in an engaging manner.
4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
5. Proofread for grammatical errors and alignment with the brand's voice.
, expected_output=A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.)

##### Task: edit

In [14]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [15]:
editor

Agent(role=Editor, goal=Edit a given blog post to align with the writing style of the organization. , backstory=You are an editor who receives a blog post from the Content Writer. Your goal is to review the blog post to ensure that it follows journalistic best practices,provides balanced viewpoints when providing opinions or assertions, and also avoids major controversial topics or opinions when possible.)

#### Creating the Crew
- Let's create our crew of agents
* Let's pass the tasks to be performed by those agents
- `verbose=2` allows you to see all the logs of the execution


In [16]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [17]:
crew

Crew(id=7acde062-cc77-44e9-ad7a-ca5093fb3775, process=sequential, number_of_agents=3, number_of_tasks=3)

#### Running the Crew
*Note*: LLMs can provide different outputs for the same input. We call fuzzy output.

In [19]:
result = crew.kickoff(inputs={"topic": "MQL5 for Expert Advisor"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on MQL5 for Expert Advisor.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points,and a call to action. 
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: MQL5 for Expert Advisor

Introduction:
- Brief overview of MQL5 and its importance in developing Expert Advisors
- Highlight the latest trends and advancements in the MQL5 platform

Key Points:
1. Latest Trends in MQL5:
- Integration of machine learning and AI in Expert Advisor development
- Increased focus on algorithmic trading strategies
- Enhanced user interface and usability features in MQL5

2. Key Players in MQL5:
- MetaQuotes Software Corp. as the developer of MQL5 platform
- Leading Expe


Display the results of your execution as markdown in the notebook.

In [20]:
from IPython.display import Markdown

Markdown(result)

# MQL5 for Expert Advisor: Revolutionizing Algorithmic Trading

## Introduction
In the realm of algorithmic trading, MQL5 serves as a cornerstone in empowering traders to create sophisticated Expert Advisors. Developed by MetaQuotes Software Corp., MQL5 is a robust programming language that enables traders to automate their trading strategies with precision. The platform has played a pivotal role in reshaping the landscape of algorithmic trading, offering advanced features and functionalities. This blog post aims to explore the latest trends and advancements in MQL5, emphasizing its significance in the development of Expert Advisors.

## Latest Trends in MQL5
A notable trend in MQL5 is the integration of machine learning and artificial intelligence in Expert Advisor development. Traders now have the opportunity to harness cutting-edge technologies to build intelligent trading systems that can adapt dynamically to market conditions. Furthermore, there is a growing emphasis on algorithmic trading strategies, leading traders to explore innovative approaches to enhance their trading performance. MQL5 has also improved its user interface and usability features, catering to traders of all levels.

## Key Players in MQL5
MetaQuotes Software Corp. stands out as the primary developer of the MQL5 platform, consistently innovating to meet the changing needs of traders. Prominent Expert Advisor developers and traders actively utilize MQL5 for trading automation, pushing the boundaries of what can be achieved in algorithmic trading. Online communities and forums dedicated to sharing MQL5 strategies and tips serve as valuable resources for traders looking to expand their expertise in the field.

## Noteworthy News on MQL5
Recent updates and releases in the MQL5 platform have introduced exciting new features and functionalities, further enhancing its capabilities. Success stories of traders leveraging MQL5 for Expert Advisor development demonstrate the platform's effectiveness in optimizing trading performance. Regulatory changes affecting the use of MQL5 in trading environments underscore the importance of staying informed and compliant in the dynamic landscape of algorithmic trading.

## Conclusion
In conclusion, MQL5 continues to drive innovation in algorithmic trading, equipping traders with advanced tools to automate their trading strategies efficiently. By keeping up with the latest trends, key players, and significant developments in the MQL5 industry, traders can unlock new opportunities and achieve greater success in their trading pursuits. Whether you are a seasoned trader seeking to automate your strategies or an enthusiast of algorithmic trading looking for advanced tools, MQL5 offers a solid platform for growth and innovation in the realm of trading automation.

*Join the MQL5 community today and discover the advantages of utilizing MQL5 for Expert Advisor development. Engage in online forums and resources to enhance your algorithmic trading skills and stay ahead of the curve in the ever-evolving world of trading automation.*